In [1]:
import json
import pandas as pd
import numpy as np
from utils.data_helper import get_markable_dataframe, get_embedding_variables
from model_builders.coreference_classifier import CoreferenceClassifierModelBuilder
from functools import reduce
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
from utils.clusterers import BestFirstClusterer, get_anaphora_scores_by_antecedent, ClosestFirstClusterer
from utils.scorers import MUCScorer, B3Scorer, AverageScorer
from utils.data_structures import UFDS

In [2]:
embedding_indexes_file_path = 'helper_files/embedding/embedding_indexes.txt'
indexed_embedding_file_path = 'helper_files/embedding/indexed_embedding.txt'

word_vector, embedding_matrix, idx_by_word, word_by_idx = get_embedding_variables(embedding_indexes_file_path, indexed_embedding_file_path)

In [3]:
markables = get_markable_dataframe("data/testing/markables_with_predicted_singleton.csv", word_vector, idx_by_word)
singletons = set(markables[markables['is_singleton'].map(lambda x: True if x[1] > 0 else False)]['id'])
markables.head()

,id,text,is_pronoun,entity_type,is_proper_name,is_first_person,previous_words,next_words,is_singleton
0,1916,"[1263, 1264, 1968, 1395]",0,"[0, 0, 0, 0, 1, 0, 1, 0, 0, 0]",1,0,[],"[999, 379, 1161, 213, 27, 1263, 1969, 1188, 14...","[0.0, 1.0]"
1,1917,[213],1,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]",0,0,"[1263, 1264, 1968, 1395, 999, 379, 1161]","[27, 1263, 1969, 1188, 1470, 25, 1161, 63, 424...","[1.0, 0.0]"
2,1918,"[1263, 1969, 1188]",0,"[0, 0, 1, 0, 1, 0, 0, 0, 0, 0]",1,0,"[1263, 1264, 1968, 1395, 999, 379, 1161, 213, 27]","[1470, 25, 1161, 63, 424, 1223, 25, 1415, 1161...","[0.0, 1.0]"
3,1919,"[1470, 25, 1161]",0,"[0, 0, 0, 0, 0, 1, 1, 0, 0, 0]",0,0,"[1968, 1395, 999, 379, 1161, 213, 27, 1263, 19...","[63, 424, 1223, 25, 1415, 1161, 876, 344, 213,...","[0.0, 1.0]"
4,1920,[424],0,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]",0,0,"[1161, 213, 27, 1263, 1969, 1188, 1470, 25, 11...","[1223, 25, 1415, 1161, 876, 344, 213, 406, 122...","[0.0, 1.0]"


In [4]:
pairs = pd.read_csv("data/testing/mention_pairs.csv")

label = np.vstack(to_categorical(pairs.is_coreference, num_classes=2))
label_chains = ClosestFirstClusterer().get_chains(get_anaphora_scores_by_antecedent(pairs.m1_id, pairs.m2_id, label))

pairs.head()

,m1_id,m2_id,is_exact_match,is_words_match,is_substring,is_abbreviation,is_appositive,is_nearest_candidate,sentence_distance,word_distance,markable_distance,is_coreference
0,1916,1917,0,0,0,0,0,1,0,3,1,1
1,1916,1918,0,0,0,0,0,0,0,5,2,0
2,1916,1919,0,0,0,0,0,0,0,8,3,0
3,1916,1920,0,0,0,0,0,0,0,12,4,0
4,1916,1921,0,0,0,0,0,0,0,13,5,0


In [5]:
max_text_length = 10
max_prev_words_length = 10
max_next_words_length = 10

def get_data(markable_ids):
    indices = reduce(lambda a, b: a + [b], map(lambda a: markables.index[markables['id'] == a].tolist()[0], markable_ids), [])
    data = markables.loc[indices]
    
    data_text = pad_sequences(data.text, maxlen=max_text_length, padding='post')
    data_previous_words = pad_sequences(data.previous_words.map(lambda seq: seq[(-1*max_prev_words_length):]), maxlen=max_prev_words_length, padding='pre')
    data_next_words = pad_sequences(data.next_words.map(lambda seq: seq[:max_next_words_length]), maxlen=max_next_words_length, padding='post')
    data_syntactic = data[['is_pronoun', 'entity_type', 'is_proper_name', 'is_first_person']]

    data_syntactic = np.array(list(map(lambda p: reduce(lambda x,y: x + y, [i if type(i) is list else [i] for i in p]), data_syntactic.values)))
    is_singleton = np.vstack(data.is_singleton)
    
    return data_text, data_previous_words, data_next_words, data_syntactic, is_singleton

def get_pair_data(markable_ids_1, markable_ids_2):
    text_1, prev_1, next_1, syntactic_1, is_singleton_1 = get_data(markable_ids_1)
    text_2, prev_2, next_2, syntactic_2, is_singleton_2 = get_data(markable_ids_2)
    
    return text_1, text_2, prev_1, prev_2, next_1, next_2, syntactic_1, syntactic_2, is_singleton_1, is_singleton_2

def get_relation_data(mention_pairs):
    return mention_pairs[['is_exact_match', 'is_words_match', 'is_substring', 'is_abbreviation', 'is_appositive', 'is_nearest_candidate', 'sentence_distance', 'word_distance', 'markable_distance']]

# Compute Baseline Score

In [6]:
baseline_result_file_path = 'baseline/test_result.txt'

baseline_ufds = UFDS()

for m1, m2 in zip(pairs.m1_id, pairs.m2_id):
    baseline_ufds.init_id(m1, m2)
    
for line in open(baseline_result_file_path, 'r').readlines():
    line = line.split(', ')
    baseline_ufds.join(int(line[0]), int(line[1]))

baseline_chains = baseline_ufds.get_chain_list()

print('MUC: ', MUCScorer().get_scores(baseline_chains, label_chains))
print('B3: ', B3Scorer().get_scores(baseline_chains, label_chains))
print('Average: ', AverageScorer([MUCScorer(), B3Scorer()]).get_scores(baseline_chains, label_chains))

MUC:  (0.5544554455445545, 0.7272727272727273, 0.6292134831460674)
B3:  (0.3124361294443262, 0.6732829670329671, 0.4268110965737344)
Average:  (0.5280122898599009, 0.5280122898599009, 0.5280122898599009)


# Test Models

In [7]:
text_1, text_2, prev_1, prev_2, next_1, next_2, syntactic_1, syntactic_2, is_singleton_1, is_singleton_2 = get_pair_data(pairs.m1_id, pairs.m2_id)
relation = get_relation_data(pairs)

In [8]:
models = {}

def get_model(features, data_generation, epoch):
    name = '_'.join([*features, data_generation, str(epoch)])
    
    if name not in models:
        models[name] = load_model(f'models/coreference_classifiers/{name}.model')
    
    return models[name]

In [9]:
base_thresholds = [0.1, 0.01, 0.001, 0.0001, 0.00001]
thresholds = [0] + [base * multiplier for base in base_thresholds for multiplier in range(1, 10)]

muc_scorer = MUCScorer()
b3_scorer = B3Scorer()
average_scorer = AverageScorer([muc_scorer, b3_scorer])

def get_sorted_scores(clusterer, pred):
    scores = [] # will be a tuple (average_f1, (prec_muc, rec_muc, f1_muc), (prec_b3, rec_b3, f1_b3), threshold)
    
    for threshold in thresholds:
        predicted_chains = clusterer.get_chains(pred, threshold)
        
#         avg_f1 = average_scorer.get_scores(predicted_chains, label_chains)[2]
        muc = muc_scorer.get_scores(predicted_chains, label_chains)
        b3 = b3_scorer.get_scores(predicted_chains, label_chains)
        avg_f1 = (muc[2] + b3[2]) / 2
        
        scores.append((avg_f1, muc, b3, threshold))
    
    return sorted(scores, reverse=True)

def reorder_score(score):
    avg_f1, muc, b3, threshold = score
    return muc, b3, avg_f1, threshold

def evaluate(features, data_generation, epoch):
    model = get_model(features, data_generation, epoch)
    
    test_features = []
    if 'words' in features:
        test_features.extend([text_1, text_2])
    if 'context' in features:
        test_features.extend([prev_1, prev_2, next_1, next_2])
    if 'syntactic' in features:
        test_features.extend([syntactic_1, syntactic_2, relation])
    
    print('getting anaphora scores by antecedent dict')
    raw_pred = model.predict(test_features, verbose=1)
    pred_without_singleton_classifier = get_anaphora_scores_by_antecedent(pairs.m1_id, pairs.m2_id, raw_pred)
    pred_with_singleton_classifier = get_anaphora_scores_by_antecedent(pairs.m1_id, pairs.m2_id, raw_pred, singletons)
    
    print('get sorted_scores_without_sc_best')
    sorted_scores_without_sc_best = get_sorted_scores(BestFirstClusterer(), pred_without_singleton_classifier)
    print('Without singleton classifier, best-first:', reorder_score(sorted_scores_without_sc_best[0]))
    
    print()
    
    print('get sorted_scores_without_sc_closest')
    sorted_scores_without_sc_closest = get_sorted_scores(ClosestFirstClusterer(), pred_without_singleton_classifier)
    print('Without singleton classifier, closest-first:', reorder_score(sorted_scores_without_sc_closest[0]))
    
    print()
    
    print('get sorted_scores_with_sc_best')
    sorted_scores_with_sc_best = get_sorted_scores(BestFirstClusterer(), pred_with_singleton_classifier)
    print('With singleton classifier, best-first:', reorder_score(sorted_scores_with_sc_best[0]))
    
    print()
    
    print('get sorted_scores_with_sc_closest')
    sorted_scores_with_sc_closest = get_sorted_scores(ClosestFirstClusterer(), pred_with_singleton_classifier)
    print('With singleton classifier, closest-first:', reorder_score(sorted_scores_with_sc_closest[0]))

## Budi

### Words + Context + Syntactic

In [14]:
evaluate(['words', 'context', 'syntactic'], 'budi', 5)

W0422 10:40:33.016038 139724918880064 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/initializers.py:111: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0422 10:40:33.021682 139724918880064 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/initializers.py:135: calling RandomNormal.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0422 10:40:33.022759 139724918880064 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:96: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) 

getting anaphora scores by antecedent dict
23689/23689==============================] - 7s 295us/sample
get sorted_scores_without_sc_best
Without singleton classifier, best-first: ((0.625, 0.6493506493506493, 0.6369426751592356), (0.5052121212121212, 0.6059752747252747, 0.5510250657366206), 0.5939838704479281, 0.30000000000000004)

get sorted_scores_without_sc_closest
Without singleton classifier, closest-first: ((0.6125, 0.6363636363636364, 0.6242038216560509), (0.4927121212121212, 0.5899496336996337, 0.5369642625866148), 0.5805840421213329, 0.30000000000000004)

get sorted_scores_with_sc_best
With singleton classifier, best-first: ((0.5977011494252874, 0.6753246753246753, 0.6341463414634148), (0.45476426481099375, 0.6173992673992673, 0.5237468268574018), 0.5789465841604082, 0.09)

get sorted_scores_with_sc_closest
With singleton classifier, closest-first: ((0.6376811594202898, 0.5714285714285714, 0.6027397260273972), (0.5334075100929033, 0.5156364468864468, 0.5243714553941523), 0.563

In [15]:
evaluate(['words', 'context', 'syntactic'], 'budi', 10)

getting anaphora scores by antecedent dict
23689/23689==============================] - 7s 305us/sample
get sorted_scores_without_sc_best
Without singleton classifier, best-first: ((0.6631578947368421, 0.8181818181818182, 0.7325581395348837), (0.48990662115662115, 0.7725045787545787, 0.5995750164958862), 0.6660665780153849, 0.2)

get sorted_scores_without_sc_closest
Without singleton classifier, closest-first: ((0.6210526315789474, 0.7662337662337663, 0.686046511627907), (0.46143828623167465, 0.7100045787545787, 0.5593500218058001), 0.6226982667168535, 0.2)

get sorted_scores_with_sc_best
With singleton classifier, best-first: ((0.6986301369863014, 0.6623376623376623, 0.68), (0.5461981566820278, 0.6061584249084249, 0.5746183423282252), 0.6273091711641126, 0.2)

get sorted_scores_with_sc_closest
With singleton classifier, closest-first: ((0.684931506849315, 0.6493506493506493, 0.6666666666666666), (0.5372375832053252, 0.5805173992673993, 0.5580395873541433), 0.612353127010405, 0.2)


In [10]:
evaluate(['words', 'context', 'syntactic'], 'budi', 20)

W0423 13:30:55.572289 139771389228864 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/initializers.py:111: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0423 13:30:55.575779 139771389228864 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/initializers.py:135: calling RandomNormal.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0423 13:30:55.576625 139771389228864 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:96: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) 

getting anaphora scores by antecedent dict
23689/23689==============================] - 5s 205us/sample
get sorted_scores_without_sc_best
Without singleton classifier, best-first: ((0.7297297297297297, 0.7012987012987013, 0.7152317880794701), (0.5996616541353383, 0.649290293040293, 0.6234899461409118), 0.6693608671101909, 0.2)

get sorted_scores_without_sc_closest
Without singleton classifier, closest-first: ((0.7230769230769231, 0.6103896103896104, 0.6619718309859155), (0.6229617604617604, 0.5420558608058609, 0.579699512096331), 0.6208356715411232, 0.30000000000000004)

get sorted_scores_with_sc_best
With singleton classifier, best-first: ((0.7666666666666667, 0.5974025974025974, 0.6715328467153285), (0.6473176612417119, 0.5378891941391941, 0.5875517401482988), 0.6295422934318137, 0.2)

get sorted_scores_with_sc_closest
With singleton classifier, closest-first: ((0.7636363636363637, 0.5454545454545454, 0.6363636363636364), (0.6526544401544402, 0.4713598901098902, 0.5473864823744434), 

## Gilang

### Words + Context + Syntactic

In [10]:
evaluate(['words', 'context', 'syntactic'], 'gilang', 5)

W0421 16:02:50.019941 140507069343552 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/initializers.py:111: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0421 16:02:50.025677 140507069343552 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/initializers.py:135: calling RandomNormal.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0421 16:02:50.026860 140507069343552 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:96: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) 

getting anaphora scores by antecedent dict
23689/23689==============================] - 7s 301us/sample
get sorted_scores_without_sc_best
Without singleton classifier, best-first: ((0.5740740740740741, 0.8051948051948052, 0.6702702702702704), (0.3637670291213599, 0.7611950549450549, 0.492279104586934), 0.5812746874286022, 0.5)

get sorted_scores_without_sc_closest
Without singleton classifier, closest-first: ((0.5092592592592593, 0.7142857142857143, 0.5945945945945946), (0.3512658945992279, 0.6641483516483516, 0.45950244592388584), 0.5270485202592402, 0.5)

get sorted_scores_with_sc_best
With singleton classifier, best-first: ((0.627906976744186, 0.7012987012987013, 0.6625766871165645), (0.4442493590551843, 0.6406364468864469, 0.5246678117789049), 0.5936222494477347, 0.5)

get sorted_scores_with_sc_closest
With singleton classifier, closest-first: ((0.5934065934065934, 0.7012987012987013, 0.6428571428571428), (0.3973474827641494, 0.6486492673992673, 0.49281062021969335), 0.567833881538

In [11]:
evaluate(['words', 'context', 'syntactic'], 'gilang', 10)

getting anaphora scores by antecedent dict
23689/23689==============================] - 8s 352us/sample
get sorted_scores_without_sc_best
Without singleton classifier, best-first: ((0.6428571428571429, 0.7012987012987013, 0.6708074534161491), (0.4756504013785567, 0.6373397435897435, 0.544750384760329), 0.607778919088239, 0.6000000000000001)

get sorted_scores_without_sc_closest
Without singleton classifier, closest-first: ((0.6190476190476191, 0.6753246753246753, 0.6459627329192547), (0.4956375442739079, 0.6141025641025641, 0.5485469697033762), 0.5972548513113154, 0.6000000000000001)

get sorted_scores_with_sc_best
With singleton classifier, best-first: ((0.6714285714285714, 0.6103896103896104, 0.6394557823129251), (0.5435910832462556, 0.5352106227106227, 0.5393683019922841), 0.5894120421526046, 0.6000000000000001)

get sorted_scores_with_sc_closest
With singleton classifier, closest-first: ((0.6428571428571429, 0.5844155844155844, 0.6122448979591837), (0.5390109890109891, 0.5174679487

In [12]:
evaluate(['words', 'context', 'syntactic'], 'gilang', 20)

getting anaphora scores by antecedent dict
23689/23689==============================] - 8s 333us/sample
get sorted_scores_without_sc_best
Without singleton classifier, best-first: ((0.5454545454545454, 0.8571428571428571, 0.6666666666666665), (0.36443805818805813, 0.8163690476190477, 0.5039196478679079), 0.5852931572672873, 0.4)

get sorted_scores_without_sc_closest
Without singleton classifier, closest-first: ((0.5617977528089888, 0.6493506493506493, 0.6024096385542169), (0.43300238408934066, 0.5999542124542124, 0.5029864859886461), 0.5526980622714315, 0.8)

get sorted_scores_with_sc_best
With singleton classifier, best-first: ((0.6022727272727273, 0.6883116883116883, 0.6424242424242423), (0.4506319811875368, 0.6214056776556777, 0.5224168560371731), 0.5824205492307077, 0.4)

get sorted_scores_with_sc_closest
With singleton classifier, closest-first: ((0.5806451612903226, 0.7012987012987013, 0.6352941176470589), (0.3904495654495655, 0.6528159340659341, 0.48864205304013175), 0.561968085

## Soon

### Words + Context + Syntactic

In [13]:
evaluate(['words', 'context', 'syntactic'], 'soon', 5)

getting anaphora scores by antecedent dict
23689/23689==============================] - 8s 339us/sample
get sorted_scores_without_sc_best
Without singleton classifier, best-first: ((0.35384615384615387, 0.2987012987012987, 0.32394366197183094), (0.35493827160493824, 0.2496794871794872, 0.2931465520062304), 0.3085451069890307, 0.7000000000000001)

get sorted_scores_without_sc_closest
Without singleton classifier, closest-first: ((0.24615384615384617, 0.2077922077922078, 0.22535211267605634), (0.29954954954954954, 0.1980769230769231, 0.23846743028235215), 0.23190977147920425, 0.7000000000000001)

get sorted_scores_with_sc_best
With singleton classifier, best-first: ((0.5747126436781609, 0.6493506493506493, 0.6097560975609756), (0.4032904202715524, 0.5853479853479854, 0.4775562706734287), 0.5436561841172022, 0.5)

get sorted_scores_with_sc_closest
With singleton classifier, closest-first: ((0.5544554455445545, 0.7272727272727273, 0.6292134831460674), (0.3326109076109076, 0.675892857142857

In [14]:
evaluate(['words', 'context', 'syntactic'], 'soon', 10)

getting anaphora scores by antecedent dict
23689/23689==============================] - 8s 356us/sample
get sorted_scores_without_sc_best
Without singleton classifier, best-first: ((0.3867924528301887, 0.5324675324675324, 0.4480874316939891), (0.30092681367849156, 0.4868360805860806, 0.37194448121668316), 0.4100159564553362, 0.9)

get sorted_scores_without_sc_closest
Without singleton classifier, closest-first: ((0.19811320754716982, 0.2727272727272727, 0.22950819672131148), (0.23658503401360545, 0.2870650183150183, 0.259391885172786), 0.24445004094704875, 0.9)

get sorted_scores_with_sc_best
With singleton classifier, best-first: ((0.5578947368421052, 0.6883116883116883, 0.616279069767442), (0.36723939777037123, 0.6342261904761904, 0.4651439789328629), 0.5407115243501525, 0.2)

get sorted_scores_with_sc_closest
With singleton classifier, closest-first: ((0.5591397849462365, 0.6753246753246753, 0.6117647058823529), (0.38097728230471595, 0.6296474358974359, 0.4747189824627964), 0.543241

In [15]:
evaluate(['words', 'context', 'syntactic'], 'soon', 20)

getting anaphora scores by antecedent dict
23689/23689==============================] - 8s 339us/sample
get sorted_scores_without_sc_best
Without singleton classifier, best-first: ((0.38392857142857145, 0.5584415584415584, 0.455026455026455), (0.31114030896639594, 0.5032967032967033, 0.3845500374064958), 0.41978824621647537, 0.6000000000000001)

get sorted_scores_without_sc_closest
Without singleton classifier, closest-first: ((0.2, 0.24675324675324675, 0.22093023255813954), (0.24140786749482399, 0.24919871794871792, 0.24524143322722294), 0.23308583289268126, 0.9)

get sorted_scores_with_sc_best
With singleton classifier, best-first: ((0.6567164179104478, 0.5714285714285714, 0.6111111111111112), (0.5719426406926407, 0.5035027472527472, 0.5355449827348352), 0.5733280469229731, 0.4)

get sorted_scores_with_sc_closest
With singleton classifier, closest-first: ((0.6125, 0.6363636363636364, 0.6242038216560509), (0.46921263554926923, 0.5863782051282052, 0.5212930094775621), 0.572748415566806

# Saving Best Model

In [12]:
def get_markable_text(idx):
    return [word_by_idx[x] for x in markables[markables['id'] == idx].text.values[0]]

In [15]:
best_model = models['words_context_syntactic_budi_20']
raw_pred_best = best_model.predict([text_1, text_2, prev_1, prev_2, next_1, next_2, syntactic_1, syntactic_2, relation], verbose=1)
pred_best_wo_sc = get_anaphora_scores_by_antecedent(pairs.m1_id, pairs.m2_id, raw_pred_best)
pred_best_w_sc = get_anaphora_scores_by_antecedent(pairs.m1_id, pairs.m2_id, raw_pred_best, singletons)
pred_chains_best_wo_sc = BestFirstClusterer().get_chains(pred_best_wo_sc, 0.2)
pred_chains_best_w_sc = BestFirstClusterer().get_chains(pred_best_w_sc, 0.2)

23689/23689==============================] - 5s 192us/sample


In [20]:
def convert_chains(chains):
    return [[(id, ' '.join(get_markable_text(id))) for id in chain] for chain in chains if len(chain) > 1]

def save_chains(chains, file_path):
    chains = convert_chains(chains)
    
    with open(file_path, 'w') as f:
        f.write(json.dumps(chains, indent=4))

In [25]:
convert_chains(pred_chains_best_wo_sc)

[[(2252, 'deputi gubernur'),
  (2255, 'hartadi a sarwono'),
  (2258, 'ia'),
  (2266, 'hartadi'),
  (2270, 'hartadi'),
  (2278, 'nya'),
  (2286, 'hartadi'),
  (2290, 'nya'),
  (2303, 'ia')],
 [(2254, 'bi'), (2268, 'bi')],
 [(2316, 'ansari'), (2323, 'ansari')],
 [(2330, 'menteri keuangan sri mulyani'),
  (2337, 'sri mulyani'),
  (2367, 'mulyani'),
  (2375, 'mulyani')],
 [(2393, 'direktur aali'), (2394, 'santosa'), (2399, 'nya')],
 [(2423, 'deputi senior'),
  (2426, 'miranda s goeltom'),
  (2436, 'miranda'),
  (2450, 'miranda')],
 [(2469, 'deputi gubernur senior bi'),
  (2470, 'miranda s goeltom'),
  (2480, 'miranda'),
  (2488, 'nya'),
  (2489, 'itung-itung bi')],
 [(2510, 'bank mandiri'),
  (2512, 'ti'),
  (2517, 'nya'),
  (2520, 'kami'),
  (2521, 'nya'),
  (2524, 'direktur teknologi dan operasional bank mandiri'),
  (2525, 'sasmita'),
  (2528, 'dia'),
  (2530, 'bank mandiri'),
  (2535, 'ia'),
  (2543, 'kita'),
  (2545, 'nya'),
  (2546, 'bank mandiri'),
  (2551, 'nya')],
 [(2565, 'menter

In [26]:
convert_chains(pred_chains_best_w_sc)

[[(2254, 'bi'), (2268, 'bi')],
 [(2255, 'hartadi a sarwono'),
  (2258, 'ia'),
  (2266, 'hartadi'),
  (2270, 'hartadi'),
  (2278, 'nya'),
  (2286, 'hartadi'),
  (2290, 'nya'),
  (2303, 'ia')],
 [(2316, 'ansari'), (2323, 'ansari')],
 [(2330, 'menteri keuangan sri mulyani'),
  (2337, 'sri mulyani'),
  (2367, 'mulyani'),
  (2375, 'mulyani')],
 [(2393, 'direktur aali'), (2394, 'santosa'), (2399, 'nya')],
 [(2426, 'miranda s goeltom'), (2436, 'miranda'), (2450, 'miranda')],
 [(2469, 'deputi gubernur senior bi'),
  (2470, 'miranda s goeltom'),
  (2480, 'miranda'),
  (2488, 'nya'),
  (2489, 'itung-itung bi')],
 [(2510, 'bank mandiri'),
  (2517, 'nya'),
  (2521, 'nya'),
  (2524, 'direktur teknologi dan operasional bank mandiri'),
  (2528, 'dia'),
  (2530, 'bank mandiri'),
  (2535, 'ia'),
  (2545, 'nya'),
  (2546, 'bank mandiri'),
  (2551, 'nya')],
 [(2565, 'menteri keuangan sri mulyani indrawati'),
  (2592, 'nya'),
  (2603, 'menko perekonomian boediono'),
  (2620, 'nya'),
  (2621, 'boediono'),


In [27]:
convert_chains(baseline_chains)

[[(2252, 'deputi gubernur'), (2253, 'bank indonesia')],
 [(2255, 'hartadi a sarwono'),
  (2258, 'ia'),
  (2266, 'hartadi'),
  (2269, 'jakarta'),
  (2270, 'hartadi'),
  (2286, 'hartadi'),
  (1927, 'kami'),
  (1933, 'sekretaris perusahaan astra otoparts'),
  (1934, 'kartina rahayu'),
  (1937, 'dia'),
  (1940, 'dia'),
  (1946, 'nya'),
  (1952, 'nya'),
  (1953, 'menkeu sri mulyani indrawati'),
  (1964, 'dia')],
 [(2303, 'ia'),
  (2311,
   'dirjend industri logam mesin tekstil departemen perindustrian ansari bukhari'),
  (2316, 'ansari'),
  (2323, 'ansari')],
 [(2330, 'menteri keuangan sri mulyani'),
  (2337, 'sri mulyani'),
  (2367, 'mulyani'),
  (2383, 'nya')],
 [(2339, 'indonesia investor forum'), (2368, 'indonesia investor forum')],
 [(2393, 'direktur aali'),
  (2394, 'santosa'),
  (2399, 'nya'),
  (2405, 'nya'),
  (2423, 'deputi senior'),
  (2424, 'bank indonesia'),
  (2426, 'miranda s goeltom'),
  (2436, 'miranda'),
  (2450, 'miranda'),
  (2460, 'kami'),
  (2466, 'nya'),
  (2469, 'dep

In [28]:
convert_chains(label_chains)

[[(2255, 'hartadi a sarwono'),
  (2258, 'ia'),
  (2266, 'hartadi'),
  (2270, 'hartadi'),
  (2286, 'hartadi'),
  (2303, 'ia')],
 [(2311,
   'dirjend industri logam mesin tekstil departemen perindustrian ansari bukhari'),
  (2316, 'ansari'),
  (2323, 'ansari')],
 [(2330, 'menteri keuangan sri mulyani'),
  (2337, 'sri mulyani'),
  (2367, 'mulyani'),
  (2375, 'mulyani')],
 [(2333, 'pdb'), (2343, 'pdb'), (2347, 'pdb')],
 [(2393, 'direktur aali'), (2394, 'santosa'), (2399, 'nya')],
 [(2426, 'miranda s goeltom'),
  (2436, 'miranda'),
  (2450, 'miranda'),
  (2464, 'nya'),
  (2466, 'nya')],
 [(2469, 'deputi gubernur senior bi'),
  (2470, 'miranda s goeltom'),
  (2480, 'miranda'),
  (2488, 'nya')],
 [(2510, 'bank mandiri'),
  (2517, 'nya'),
  (2520, 'kami'),
  (2530, 'bank mandiri'),
  (2546, 'bank mandiri'),
  (2554, 'nya')],
 [(2524, 'direktur teknologi dan operasional bank mandiri'),
  (2525, 'sasmita'),
  (2528, 'dia'),
  (2535, 'ia'),
  (2545, 'nya')],
 [(2565, 'menteri keuangan sri mulyani

In [21]:
save_chains(pred_chains_best_w_sc, 'result/with_singleton_classifier.json')

In [22]:
save_chains(pred_chains_best_wo_sc, 'result/without_singleton_classifier.json')